In [7]:
import pysam
import pandas as pd
from Bio import SeqIO
from Bio.SeqIO import FastaIO

Name = 'E14_Lib1'

# Sort into Bam file
pysam.sort("-o", Name +".bam", Name + ".sam")
pysam.index(Name + '.bam')

# count all the on target reads
bamfile = pysam.AlignmentFile(Name + '.bam', "rb")
Data = bamfile.get_index_statistics()
df = pd.DataFrame(Data)
OnTar = df.mapped//2
print(sum(OnTar))

# Write On-Target plex to Excel
with pd.ExcelWriter(Name+'_OnTarget.xlsx') as writer: 
    OnTar.to_excel(writer, index = False, header=False)

# write out all the unmapped reads
mark1 = 0
mark2 = 0
samfile = pysam.AlignmentFile(Name+'.sam', 'r')
with open(Name + '_unmapped_R1.fasta', 'w') as f1:
    with open(Name + '_unmapped_R2.fasta', 'w') as f2:
            for read in samfile.fetch():
                if read.is_paired and read.is_unmapped:
                    if read.is_read1:
                        read1 = read
                        mark1 = 1
                    elif read.is_read2:
                        read2 = read
                        mark2 = 1
                        
                        if mark1 == 1 and mark2 == 1:
                            f1.write('>%s\n' % read1.query_name)
                            f1.write('%s\n' % read1.query_sequence)   

                            f2.write('>%s\n' % read2.query_name)
                            f2.write('%s\n' % read2.query_sequence) 

                            mark1 = 0
                            mark2 = 0
                            
# combine all the short reads and unmapped reads in a fasta file
# define file_list here
fList1 = [Name + '_QC_R1_short.fasta',Name + '_unmapped_R1.fasta']
fList2 = [Name + '_QC_R2_short.fasta',Name + '_unmapped_R2.fasta']


with open(Name + '_DimerCand_R1.fasta', 'w') as w_file:
    for filen in fList1:
        with open(filen, 'r') as o_file:
            for record in SeqIO.parse(o_file, 'fasta'):
                w_file.write('>%s\n' % record.id) 
                w_file.write('%s\n' % record.seq)  
                
with open(Name + '_DimerCand_R2.fasta', 'w') as w_file:
    for filen in fList2:
        with open(filen, 'r') as o_file:
            for record in SeqIO.parse(o_file, 'fasta'):
                w_file.write('>%s\n' % record.id) 
                w_file.write('%s\n' % record.seq)    
                            
                            

36203
